In [ ]:
final_lags=[1,2,3,4,5,10,11] # este código funciona para reemplazar las primeras predicciones del mes 8 año 2016
index=list(range(1,12,1))
meses=pd.DataFrame([9,10,11,12,1,2,3,4,5,6,7],columns=['Meses'],index=index)
meses=meses.loc[final_lags,:]
meses=meses.Meses.values.tolist()
for i,j in zip(final_lags,meses):
    if i<5:# para separar 2016 y 2017
        año=2016
    else:
        año=2017
    rows_toreplace=X_test[(X_test.Año==año) & (X_test.Mes==j)].index.values # filas a reemplazar con predicciones
    X_test.loc[rows_toreplace,'t-'+str(i)]=dtest_predictions #reemplazando los datos reales por las predicciones en conjunto test


In [ ]:
final_lags=[1,2,3,4,5,10,11] #rezagos significativos 
total_lags=[1,2,3,4,5,6,7,8,9,10,11]#todos los rezagos 
meses_apredecir=[8,9,10,11,12,1,2,3,4,5,6] #meses a predecir en el ciclo for(11 meses) el 12° mes se predice por separado al último
aux=0 #se utiliza para ir disminuyendo el umbral para saber si el mes a reemplazar pertenece a 2016 o 2017 a partir del mes 12 2016 todos los 
#reemplazos se harán en el año 2017
resultados=pd.DataFrame(data=np.zeros((420,1)),columns=['Prediccion']) #Data frame donde se guardarán los pronósticos 
for t,mes_predic in zip(range(1,12,1),meses_apredecir):  ########aureca , es el bueno por fin!
    if t<6:
        año_predic=2016
    else:
        año_predic=2017
    dtest_predictions = gbm0.predict(X_test[(X_test.Año==año_predic) & (X_test.Mes==mes_predic)].reset_index(drop=True)) #hacer predicciones
    #mes con mes 

    lags=set(total_lags).intersection(final_lags) #para encontrar los que coinciden entre total_lags y final_lags 
    lags=list(lags) #rezagos a reemplazar en cada iteración de predicciones 

    index=list(range(1,13-t,1))
    meses=pd.DataFrame([9,10,11,12,1,2,3,4,5,6,7],columns=['Meses']) #todos los meses en los que se reemplazarían las predicciones 
    meses=meses.iloc[t-1:,:]
    meses['index']=index
    meses=meses.set_index('index')
    meses=meses.loc[lags,:]
    meses=meses.Meses.values.tolist() #meses en los que se reemplaza las predicciones dados los rezagos significativos (final_lags)

    total_lags=total_lags[:-1] #para ir eliminando el último elemento de la lista "total_lags"
    
    for i,j in zip(lags,meses): #ciclo para ir reemplazando las predicciones ,"i" sería el contador para rezagos y "j" contador para meses a reemplazar
        if i<(5-aux):# para separar 2016 y 2017
            año_replace=2016
        else:
            año_replace=2017
        rows_toreplace=X_test[(X_test.Año==año_replace) & (X_test.Mes==j)].index.values # filas a reemplazar con predicciones
        X_test.loc[rows_toreplace,'t-'+str(i)]=dtest_predictions #reemplazando los datos reales por las predicciones en conjunto test
    aux=aux+1 
    print(dtest_predictions)
    resultados.Prediccion[(t-1)*35:t*35]=dtest_predictions
    
##predicciones del último mes( Julio 2017)
dtest_predictions = gbm0.predict(X_test[(X_test.Año==2017) & (X_test.Mes==7)].reset_index(drop=True)) 
resultados.Prediccion[385:]=dtest_predictions

In [ ]:
#dividir train,cross y tst
data_train=datos.query('Año<2015').reset_index(drop=True) #desde diciembre 2000 hasta 2012 
data_cross=datos.query('(Año==2016 & Mes<8) or (Año<2016 & Año>2014)').reset_index(drop=True) #2013 hasta julio 2016
data_test=datos.query('Año == 2017 or (Año==2016 & Mes>7)').reset_index(drop=True) #ultimos 12 meses

#separar "x,y"
Y_train=data_train.iloc[:,4:5]
X_train=data_train.ix[:, data_train.columns != 'Monto']

Y_cross=data_cross.iloc[:,4:5]
X_cross=data_cross.ix[:, data_cross.columns != 'Monto']

Y_test=data_test.iloc[:,4:5]
X_test=data_test.ix[:, data_test.columns != 'Monto']

#Fit the algorithm on the training data
gbm0.fit(X_train,Y_train)

#Predict test set:
#dtest_predictions = gbm0.predict(X_test.query('Año==2016 & Mes==8').reset_index(drop=True))
#dtest_predictions = gbm0.predict(X_test[(X_test.Año==2016) & (X_test.Mes==10)].reset_index(drop=True))
#dtest_predictions